In [ ]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
import nltk



# --- Load the datasets ---
try:
    news_df = pd.read_csv("../../data/raw_analyst_ratings.csv")
    historical_df = pd.read_csv("../../data/yfinance_data/AAPL_historical_data.csv")
    print("Datasets loaded successfully: 'Book1.xlsx - Sheet1.csv' (News) and 'AAPL_historical_data.csv' (Historical).")
except FileNotFoundError:
    print("Error: One or both of the dataset files were not found.")
    print("Please ensure 'Book1.xlsx - Sheet1.csv' and 'AAPL_historical_data.csv' are in the same directory as this script.")
    exit()

# --- Initial Data Inspection ---
print("\n--- News Data Info ---")
news_df.info()
print("\nNews Data Head (first 5 rows):")
print(news_df.head())

print("\n--- Historical Data Info ---")
historical_df.info()
print("\nHistorical Data Head (first 5 rows):")
print(historical_df.head())

# --- Data Preprocessing and Date Alignment ---

# Rename 'date' column in news_df to 'news_date' for clarity and to avoid conflicts
if 'date' in news_df.columns:
    news_df.rename(columns={'date': 'news_date'}, inplace=True)
    print("\nRenamed 'date' column in news_df to 'news_date'.")
else:
    print("\nWarning: 'date' column not found in news_df. Assuming 'news_date' is available.")

# Convert 'news_date' to datetime and normalize to date only (remove time component)
if 'news_date' in news_df.columns:
    news_df['news_date'] = pd.to_datetime(news_df['news_date'], errors='coerce').dt.normalize()
    news_df.dropna(subset=['news_date'], inplace=True) # Drop rows where date conversion failed
    print("Converted 'news_date' to datetime and normalized to date only.")
else:
    print("Error: 'news_date' column is missing or could not be processed in news_df.")
    exit()

# Rename 'Date' column in historical_df to 'stock_date' for clarity and to avoid conflicts
if 'Date' in historical_df.columns:
    historical_df.rename(columns={'Date': 'stock_date'}, inplace=True)
    print("Renamed 'Date' column in historical_df to 'stock_date'.")
elif 'date' in historical_df.columns: # Fallback in case it's 'date' (lowercase)
    historical_df.rename(columns={'date': 'stock_date'}, inplace=True)
    print("Renamed 'date' column in historical_df to 'stock_date'.")
else:
    print("Warning: 'Date' or 'date' column not found in historical_df. Assuming 'stock_date' is available.")

# Convert 'stock_date' to datetime
if 'stock_date' in historical_df.columns:
    historical_df['stock_date'] = pd.to_datetime(historical_df['stock_date'], errors='coerce').dt.normalize()
    historical_df.dropna(subset=['stock_date'], inplace=True) # Drop rows where date conversion failed
    historical_df.sort_values(by='stock_date', inplace=True)
    print("Converted 'stock_date' to datetime and sorted historical_df by date.")
else:
    print("Error: 'stock_date' column is missing or could not be processed in historical_df.")
    exit()

# --- Calculate Daily Stock Returns ---
if 'Close' in historical_df.columns:
    historical_df['Daily_Return'] = historical_df['Close'].pct_change() * 100
    print("\nCalculated 'Daily_Return' for historical stock data.")
else:
    print("\nError: 'Close' column not found in historical_df. Cannot calculate daily returns.")
    exit()

# --- Sentiment Analysis on News Headlines ---
def get_sentiment(text):
    if pd.isna(text):
        return None, None
    try:
        analysis = TextBlob(str(text))
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity
    except Exception as e:
        print(f"Error processing sentiment for text: '{str(text)[:50]}...' - {e}")
        return None, None

if 'headlin' in news_df.columns:
    # Apply sentiment analysis
    news_df[['Sentiment_Polarity', 'Sentiment_Subjectivity']] = news_df['headlin'].apply(
        lambda x: pd.Series(get_sentiment(x))
    )
    print("\nPerformed sentiment analysis on 'headlin' column.")
else:
    print("\nError: 'headlin' column not found in news_df. Cannot perform sentiment analysis.")
    exit()

# --- Aggregate Sentiments by Date ---
if 'news_date' in news_df.columns and 'Sentiment_Polarity' in news_df.columns:
    # Compute average daily sentiment scores
    daily_sentiment = news_df.groupby('news_date')['Sentiment_Polarity'].mean().reset_index()
    daily_sentiment.rename(columns={'Sentiment_Polarity': 'Average_Sentiment'}, inplace=True)
    print("Aggregated daily average sentiment scores.")
else:
    print("Error: Cannot aggregate sentiments. Check 'news_date' and 'Sentiment_Polarity' columns in news_df.")
    exit()

# --- Merge Dataframes ---
if 'stock_date' in historical_df.columns and 'news_date' in daily_sentiment.columns:
    # Merge historical data with aggregated daily sentiment
    merged_df = pd.merge(historical_df, daily_sentiment, left_on='stock_date', right_on='news_date', how='inner')
    print("\nMerged historical stock data with daily news sentiment.")
    print("\nMerged Data Head (first 5 rows):")
    print(merged_df.head())
else:
    print("Error: Cannot merge datasets. Check 'stock_date' in historical_df and 'news_date' in daily_sentiment.")
    exit()

# --- Correlation Analysis ---
if 'Daily_Return' in merged_df.columns and 'Average_Sentiment' in merged_df.columns:
    correlation = merged_df['Daily_Return'].corr(merged_df['Average_Sentiment'])
    print(f"\n--- Correlation Analysis Result ---")
    print(f"Pearson Correlation between Daily Stock Returns and Average News Sentiment: {correlation:.4f}")

    # --- KPIs ---
    print("\n--- KPIs ---")
    print("Sentiment Analysis: Completed. Sample sentiment for the first few headlines:")
    # Display sample sentiments
    for i in range(min(5, len(news_df))):
        headline = news_df.iloc[i]['headlin']
        polarity = news_df.iloc[i]['Sentiment_Polarity']
        subjectivity = news_df.iloc[i]['Sentiment_Subjectivity']
        if pd.isna(polarity) or pd.isna(subjectivity):
            print(f"  - Headline: {str(headline)[:70]}... (Sentiment Not Available)")
        else:
            print(f"  - Headline: {str(headline)[:70]}...")
            print(f"    Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}")

    print(f"\nCorrelation Strength: {correlation:.4f}")

    # --- Plotting for Visualization ---
    plt.figure(figsize=(12, 7))
    sns.scatterplot(x='Average_Sentiment', y='Daily_Return', data=merged_df)
    plt.title('Daily Stock Return vs. Average News Sentiment (AAPL)', fontsize=16)
    plt.xlabel('Average Daily News Sentiment (Polarity)', fontsize=12)
    plt.ylabel('Daily Stock Return (%)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.axhline(0, color='grey', linestyle='--', linewidth=0.8) # Add a line at 0 daily return
    plt.axvline(0, color='grey', linestyle='--', linewidth=0.8) # Add a line at 0 sentiment polarity
    plt.text(merged_df['Average_Sentiment'].min(), merged_df['Daily_Return'].max(),
             f'Correlation: {correlation:.4f}',
             bbox=dict(facecolor='white', alpha=0.5), fontsize=10)
    plt.show()

else:
    print("\nError: Required columns for correlation analysis (Daily_Return, Average_Sentiment) are missing from the merged DataFrame.")

[Debug] Unparsed news dates after fallback parsing: 431
